In [1]:
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By

/Users/mamoudou/Developer/Python/ProjetTrutoré/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


# CONSTS

In [2]:
CHROME_DRIVER_PATH = "./chromedriver"
BRAVE_PATH = "/Applications/Brave Browser.app/Contents/MacOS/Brave Browser"
DOWNLOAD_DIRECTORY = "/Users/mamoudou/Developer/Python/ProjetTrutoré/"
ARCHIVE_DIRECTORY = "/Users/mamoudou/Developer/Python/ProjetTrutoré/archive"
URL = "https://donneespubliques.meteofrance.fr/?fond=produit&id_produit=90&id_rubrique=32"

## Browser Configs

In [3]:
options = Options()
options.binary_location = BRAVE_PATH  

prefs = {
    "download.default_directory": DOWNLOAD_DIRECTORY,
    "download.prompt_for_download": False,
    "download.directory_upgrade": True,
    "safebrowsing.enabled": True 
}
options.add_experimental_option("prefs", prefs)

In [4]:
service = Service(executable_path=CHROME_DRIVER_PATH)

driver = webdriver.Chrome(service=service, options=options)



driver.get(URL)
print("Page Title:", driver.title)
telecharger_button = driver.find_element(By.XPATH, "//input[@type='submit' and @value='Télécharger']")
telecharger_button.click()

time.sleep(5)
driver.quit()

Page Title: Données Publiques de Météo-France - Données SYNOP essentielles OMM


## Fixing the file extension problem

In [5]:
from file_management import change_file_extension

cvs_file = change_file_extension(DOWNLOAD_DIRECTORY)

Le fichier a été renommé : /Users/mamoudou/Developer/Python/ProjetTrutoré/synop.2024123018.txt -> /Users/mamoudou/Developer/Python/ProjetTrutoré/synop.2024123018.csv


# Starting the ELT

In [6]:
import pandas as pd
import numpy as np
df = pd.read_csv(cvs_file, sep=";", dtype={"date": str})

print(df.to_string())

    numer_sta            date    pmer  tend cod_tend   dd         ff           t          td    u     vv  ww  w1  w2    n nbas  hbas  cl  cm  ch    pres niv_bar geop tend24        tn12        tn24        tx12        tx24     tminsol  sw  tw      raf10     rafper  per etat_sol  ht_neige    ssfrai perssfrai        rr1        rr3        rr6       rr12       rr24 phenspe1 phenspe2 phenspe3 phenspe4 nnuage1 ctype1 hnuage1 nnuage2 ctype2 hnuage2 nnuage3 ctype3 hnuage3 nnuage4 ctype4 hnuage4  Unnamed: 59
0        7005  20241230180000  102990   -20        6  180   1.500000  277.350000  276.950000   97   6090   0  mq  mq   mq   mq    mq  mq  mq  mq  102080      mq   mq   -180  277.350000          mq  280.050000          mq  278.250000  mq  mq   3.700000   4.900000  -10        1  0.000000        mq        mq   0.000000   0.000000   0.000000   0.000000  -0.100000       mq       mq       mq       mq      mq     mq      mq      mq     mq      mq      mq     mq      mq      mq     mq      mq        

# Récupération des informations importantes à savoir :
1. Indicatif OMM station : numéro de station `numer_sta` -> `int`
2. Date (UTC) : `date` -> `datetime`
3. Pression au niveau mer : `pmer` -> `int`
4. Variation de pression en 3 heures : `tend` -> `int`
5. Type de tendance barométrique : `cod_tend` -> `int`
6. Direction du vent moyen 10 mn : `dd` -> `int`
7. Vitesse du vent moyen 10 mn : `ff` -> `float`
8. Température : `t` -> `float`
9. Point de rosée : `td` -> `float`
10. Humidité : `u` -> `int`
11. Visibilité horizontale : `vv` -> `float`
12. Temps présent : `ww` -> `int`
13. Nébulosité totale : `n` -> `float`
14. Nébulosité des nuages de l'étage inférieur : `nbas` -> `int`
15. Hauteur de la base des nuages de l'étage inférieur : `hbas` -> `int`
16. Pression station : `pres` -> `int`
17. Variation de pression en 24 heures : `tend24` -> `int`
18. Température minimale sur N heures : `tn12` -> `float`
19. Température maximale sur N heures : `tx12` -> `float`
20. Température minimale du sol sur 12 heures : `tminsol` -> `float`
21. Rafales sur les 10 dernières minutes : `raf10` -> `float`
22. Rafales sur une période : `rafper` -> `float`
23. Période de mesure des rafales : `per` -> `float`
24. Précipitations dans les N dernières heures : `rr12` -> `float`

In [7]:
df = df[['numer_sta', 'date', "pmer", "tend", "cod_tend", "dd", "ff", "t", "td", "u", "vv", "ww", "n", "nbas", "hbas","pres", "tend24", "tn12", "tx12", "tminsol", "raf10", "rafper", "per", "rr12" ]]


print(df.to_string())


    numer_sta            date    pmer  tend cod_tend   dd         ff           t          td    u     vv  ww    n nbas  hbas    pres tend24        tn12        tx12     tminsol      raf10     rafper  per       rr12
0        7005  20241230180000  102990   -20        6  180   1.500000  277.350000  276.950000   97   6090   0   mq   mq    mq  102080   -180  277.350000  280.050000  278.250000   3.700000   4.900000  -10   0.000000
1        7015  20241230180000  102960     0        0  180   5.500000  277.950000  277.550000   97   6680   0  100    8   150  102370   -200  277.950000  279.550000  277.850000   7.300000   8.700000  -10  -0.100000
2        7020  20241230180000      mq    10        1   mq         mq          mq          mq   mq     mq  mq   mq   mq    mq  102770   -290          mq          mq          mq         mq         mq  -10         mq
3        7027  20241230180000  103040   -10        6  160   2.000000  275.750000  274.850000   94  16130   0  100    8   450  102190   -200  275

## Remplacer les valeurs manquantes `mq` par `NaN`

In [8]:
df.replace("mq", np.nan, inplace=True)
print(df.to_string())

    numer_sta            date    pmer  tend cod_tend   dd         ff           t          td    u     vv   ww    n nbas  hbas    pres tend24        tn12        tx12     tminsol      raf10     rafper  per       rr12
0        7005  20241230180000  102990   -20        6  180   1.500000  277.350000  276.950000   97   6090    0  NaN  NaN   NaN  102080   -180  277.350000  280.050000  278.250000   3.700000   4.900000  -10   0.000000
1        7015  20241230180000  102960     0        0  180   5.500000  277.950000  277.550000   97   6680    0  100    8   150  102370   -200  277.950000  279.550000  277.850000   7.300000   8.700000  -10  -0.100000
2        7020  20241230180000     NaN    10        1  NaN        NaN         NaN         NaN  NaN    NaN  NaN  NaN  NaN   NaN  102770   -290         NaN         NaN         NaN        NaN        NaN  -10        NaN
3        7027  20241230180000  103040   -10        6  160   2.000000  275.750000  274.850000   94  16130    0  100    8   450  102190   -200

## Nettoyages de colonnes non renseignées 

1. Informations générales
	-	Numéro de station : `numer_sta` -> int
	-	Date (UTC) : `date` -> datetime
2. Pression
	-	Pression au niveau mer : `pmer` -> int
	-	Variation de pression en 3 heures : `tend` -> int
	-	Type de tendance barométrique : `cod_tend` -> int
	-	Pression station : `pres` -> int
	-	Variation de pression en 24 heures : `tend24` -> int
3. Vent
	-	Direction du vent moyen 10 mn : `dd` -> int
	-	Vitesse du vent moyen 10 mn : `ff` -> float
	-	Rafales sur les 10 dernières minutes : `raf10` -> float
	-	Rafales sur une période : `rafper` -> float
	-	Période de mesure des rafales : `per` -> float
4. Température
	-	Température : `t` -> float
	-	Point de rosée : `td` -> float
	-	Température minimale sur N heures : `tn12` -> float
	-	Température maximale sur N heures : `tx12` -> float
	-	Température minimale du sol sur 12 heures : `tminsol` -> float
5. Humidité
	-	Humidité : `u` -> int
6. Visibilité
	-	Visibilité horizontale : `vv` -> float
7. Nébulosité et nuages
	-	Nébulosité totale : `n` -> float
	-	Nébulosité des nuages de l’étage inférieur : `nbas` -> int
	-	Hauteur de la base des nuages de l’étage inférieur : `hbas` -> int
8. Précipitations
	-	Précipitations dans les N dernières heures : `rr12` -> float
9. Temps présent
	-	Temps présent : `ww` -> int

In [9]:
df[['dd', 'ff', 'raf10', 'rafper', 'rr12', "per"]] = df[['dd', 'ff', 'raf10', 'rafper', 'rr12', "per"]].fillna(0)

moyenne = ["pmer", "tend", "cod_tend", "pres", "tend24", "t", "td", "tn12", "tx12", "tminsol", "u", "vv", "n", "nbas", "hbas", "ww"]
df[moyenne] = df[moyenne].apply(pd.to_numeric, errors='coerce')

for col in moyenne:
    df[col] = df[col].fillna(df[col].mean())

print(df.to_string())

    numer_sta            date           pmer        tend  cod_tend   dd         ff           t          td           u            vv         ww           n      nbas         hbas           pres      tend24      tn12        tx12     tminsol      raf10     rafper  per       rr12
0        7005  20241230180000  102990.000000  -20.000000  6.000000  180   1.500000  277.350000  276.950000   97.000000   6090.000000   0.000000   94.535714  6.055556   697.096774  102080.000000 -180.000000  277.3500  280.050000  278.250000   3.700000   4.900000  -10   0.000000
1        7015  20241230180000  102960.000000    0.000000  0.000000  180   5.500000  277.950000  277.550000   97.000000   6680.000000   0.000000  100.000000  8.000000   150.000000  102370.000000 -200.000000  277.9500  279.550000  277.850000   7.300000   8.700000  -10  -0.100000
2        7020  20241230180000  102477.735849   10.000000  1.000000    0          0  281.269643  278.662963   87.518519  14422.093023  11.880952   94.535714  6.055556 

## Typages des colonnes

In [10]:
df["date"] = pd.to_datetime(df["date"], format='%Y%m%d%H%M%S')
df['date'] = df['date'].dt.strftime("%Y-%m-%d %H:%M:%S")

entier = ["numer_sta", "pmer", "tend", "cod_tend", "dd", "u", "ww", "nbas", "hbas", "pres", "tend24"]
for i in entier:
    print(f"{i} -> {df[col].dtype}")
    df[entier] = df[entier].astype(int)
    print(f"{i} -> {df[col].dtype}")
    print("")

double = ['dd', 'ff', 'raf10', 'rafper', 'rr12', "per", "t", "td", "tn12", "tx12", "tminsol", "n", "vv"]
for i in double:
    print(f"{i} -> {df[col].dtype}")
    df[double] = df[double].astype(float)
    print(f"{i} -> {df[col].dtype}")
    print("")

numer_sta -> float64
numer_sta -> int64

pmer -> int64
pmer -> int64

tend -> int64
tend -> int64

cod_tend -> int64
cod_tend -> int64

dd -> int64
dd -> int64

u -> int64
u -> int64

ww -> int64
ww -> int64

nbas -> int64
nbas -> int64

hbas -> int64
hbas -> int64

pres -> int64
pres -> int64

tend24 -> int64
tend24 -> int64

dd -> int64
dd -> int64

ff -> int64
ff -> int64

raf10 -> int64
raf10 -> int64

rafper -> int64
rafper -> int64

rr12 -> int64
rr12 -> int64

per -> int64
per -> int64

t -> int64
t -> int64

td -> int64
td -> int64

tn12 -> int64
tn12 -> int64

tx12 -> int64
tx12 -> int64

tminsol -> int64
tminsol -> int64

n -> int64
n -> int64

vv -> int64
vv -> int64



In [11]:
print(df.to_string())

    numer_sta                 date    pmer  tend  cod_tend     dd    ff           t          td    u            vv  ww           n  nbas  hbas    pres  tend24      tn12        tx12     tminsol  raf10  rafper   per  rr12
0        7005  2024-12-30 18:00:00  102990   -20         6  180.0   1.5  277.350000  276.950000   97   6090.000000   0   94.535714     6   697  102080    -180  277.3500  280.050000  278.250000    3.7     4.9 -10.0   0.0
1        7015  2024-12-30 18:00:00  102960     0         0  180.0   5.5  277.950000  277.550000   97   6680.000000   0  100.000000     8   150  102370    -200  277.9500  279.550000  277.850000    7.3     8.7 -10.0  -0.1
2        7020  2024-12-30 18:00:00  102477    10         1    0.0   0.0  281.269643  278.662963   87  14422.093023  11   94.535714     6   697  102770    -290  274.8175  279.135366  275.580303    0.0     0.0 -10.0   0.0
3        7027  2024-12-30 18:00:00  103040   -10         6  160.0   2.0  275.750000  274.850000   94  16130.000000   0  

## Conversion des températures de Kelvin en Celcuis

In [12]:
celcuis = ["t", "td", "tn12", "tx12", "tminsol"]
for i in celcuis:
    df[i] = df[i] - 273.15

print(df.to_string())

    numer_sta                 date    pmer  tend  cod_tend     dd    ff          t         td    u            vv  ww           n  nbas  hbas    pres  tend24     tn12       tx12   tminsol  raf10  rafper   per  rr12
0        7005  2024-12-30 18:00:00  102990   -20         6  180.0   1.5   4.200000   3.800000   97   6090.000000   0   94.535714     6   697  102080    -180   4.2000   6.900000  5.100000    3.7     4.9 -10.0   0.0
1        7015  2024-12-30 18:00:00  102960     0         0  180.0   5.5   4.800000   4.400000   97   6680.000000   0  100.000000     8   150  102370    -200   4.8000   6.400000  4.700000    7.3     8.7 -10.0  -0.1
2        7020  2024-12-30 18:00:00  102477    10         1    0.0   0.0   8.119643   5.512963   87  14422.093023  11   94.535714     6   697  102770    -290   1.6675   5.985366  2.430303    0.0     0.0 -10.0   0.0
3        7027  2024-12-30 18:00:00  103040   -10         6  160.0   2.0   2.600000   1.700000   94  16130.000000   0  100.000000     8   450  10

## Arrondir tous les decimaux à 2chiffres apes la virgules

In [13]:
df[double] = df[double].round(2)
print(df.to_string())

    numer_sta                 date    pmer  tend  cod_tend     dd    ff      t     td    u        vv  ww       n  nbas  hbas    pres  tend24   tn12   tx12  tminsol  raf10  rafper   per  rr12
0        7005  2024-12-30 18:00:00  102990   -20         6  180.0   1.5   4.20   3.80   97   6090.00   0   94.54     6   697  102080    -180   4.20   6.90     5.10    3.7     4.9 -10.0   0.0
1        7015  2024-12-30 18:00:00  102960     0         0  180.0   5.5   4.80   4.40   97   6680.00   0  100.00     8   150  102370    -200   4.80   6.40     4.70    7.3     8.7 -10.0  -0.1
2        7020  2024-12-30 18:00:00  102477    10         1    0.0   0.0   8.12   5.51   87  14422.09  11   94.54     6   697  102770    -290   1.67   5.99     2.43    0.0     0.0 -10.0   0.0
3        7027  2024-12-30 18:00:00  103040   -10         6  160.0   2.0   2.60   1.70   94  16130.00   0  100.00     8   450  102190    -200   2.40   5.90     6.30    3.5     6.2 -10.0   0.0
4        7037  2024-12-30 18:00:00  103080   

# Archive file

In [14]:
from file_management import move_file

move_file(cvs_file, ARCHIVE_DIRECTORY)

Le fichier a été déplacé de /Users/mamoudou/Developer/Python/ProjetTrutoré/synop.2024123018.csv à /Users/mamoudou/Developer/Python/ProjetTrutoré/archive/synop.2024123018.csv


## Converting DF to JSON and then to readable dict

In [15]:
from datetime import datetime

In [16]:
import pprint   
import json

json_data = df.to_json(orient='records')
dict_data = json.loads(json_data)

pprint.pprint(dict_data)

[{'cod_tend': 6,
  'date': '2024-12-30 18:00:00',
  'dd': 180.0,
  'ff': 1.5,
  'hbas': 697,
  'n': 94.54,
  'nbas': 6,
  'numer_sta': 7005,
  'per': -10.0,
  'pmer': 102990,
  'pres': 102080,
  'raf10': 3.7,
  'rafper': 4.9,
  'rr12': 0.0,
  't': 4.2,
  'td': 3.8,
  'tend': -20,
  'tend24': -180,
  'tminsol': 5.1,
  'tn12': 4.2,
  'tx12': 6.9,
  'u': 97,
  'vv': 6090.0,
  'ww': 0},
 {'cod_tend': 0,
  'date': '2024-12-30 18:00:00',
  'dd': 180.0,
  'ff': 5.5,
  'hbas': 150,
  'n': 100.0,
  'nbas': 8,
  'numer_sta': 7015,
  'per': -10.0,
  'pmer': 102960,
  'pres': 102370,
  'raf10': 7.3,
  'rafper': 8.7,
  'rr12': -0.1,
  't': 4.8,
  'td': 4.4,
  'tend': 0,
  'tend24': -200,
  'tminsol': 4.7,
  'tn12': 4.8,
  'tx12': 6.4,
  'u': 97,
  'vv': 6680.0,
  'ww': 0},
 {'cod_tend': 1,
  'date': '2024-12-30 18:00:00',
  'dd': 0.0,
  'ff': 0.0,
  'hbas': 697,
  'n': 94.54,
  'nbas': 6,
  'numer_sta': 7020,
  'per': -10.0,
  'pmer': 102477,
  'pres': 102770,
  'raf10': 0.0,
  'rafper': 0.0,
  'rr

In [17]:
first = dict_data[0]
js = json.dumps(first)

print(js)

{"numer_sta": 7005, "date": "2024-12-30 18:00:00", "pmer": 102990, "tend": -20, "cod_tend": 6, "dd": 180.0, "ff": 1.5, "t": 4.2, "td": 3.8, "u": 97, "vv": 6090.0, "ww": 0, "n": 94.54, "nbas": 6, "hbas": 697, "pres": 102080, "tend24": -180, "tn12": 4.2, "tx12": 6.9, "tminsol": 5.1, "raf10": 3.7, "rafper": 4.9, "per": -10.0, "rr12": 0.0}


In [18]:
import requests


In [19]:
from api import send_data

url_api = "https://127.0.0.1:8000/product" 
donnees = {
    "name": "Produit modifié",
    "description": "Superbe desc",
    "price": 122.2,
    'createdAt': '2024-12-22T16:44:44+00:00',
    'updatedAt': '2024-12-22T16:44:44+00:00'
}

reponse = send_data(url_api, donnees, requests)


Requête réussie!
